## Imports


In [1]:
import numpy as np
import pandas as pd
from sklearn import neighbors
from sklearn.model_selection import KFold
from geopy.distance import geodesic
from IPython.display import display # Allows the use of display() for DataFrames
%matplotlib inline
from sklearn.preprocessing import StandardScaler

## Normalizando Dados (menos lat e lon)

In [2]:
fit_df = pd.read_csv("LocTreino_Equipe_8.csv")
bts_df = pd.read_csv("dados_BTSs.csv")

fit_df.dropna(inplace=True)
fit_df.reset_index(inplace=True,drop=True)

scaler = StandardScaler()
numerical = ["pathBTS1","pathBTS2","pathBTS3","pathBTS4","pathBTS5","pathBTS6","taBTS1","taBTS2","taBTS3","taBTS4","taBTS5","taBTS6"]

fit_df[numerical] = pd.DataFrame(scaler.fit_transform(fit_df[numerical]))
display(fit_df.head())

,pontoId,lat,lon,pathBTS1,pathBTS2,pathBTS3,pathBTS4,pathBTS5,pathBTS6,taBTS1,taBTS2,taBTS3,taBTS4,taBTS5,taBTS6
0,411,-8.075448,-34.893459,-0.245444,-0.119072,-0.317120,-0.050926,-0.658604,0.988673,0.492008,-1.223457,-0.551552,-1.223457,1.056149,1.386303
1,516,-8.074647,-34.890316,-0.320266,0.281964,1.004737,-0.799438,0.717571,1.750301,0.492008,-1.223457,0.511090,-1.223457,-0.125506,1.386303
2,1673,-8.069371,-34.898918,-0.380933,-0.427351,-0.686353,-0.166293,-0.105143,-1.308454,0.492008,-0.025605,-0.551552,-0.025605,1.056149,-0.055876
3,1140,-8.071290,-34.900772,0.045756,-0.029043,0.247808,0.284875,1.231143,0.285821,0.492008,-0.025605,-1.614194,-0.025605,1.056149,-0.055876
4,1043,-8.072050,-34.897167,-0.670110,-0.307313,-0.066041,-0.394280,0.565493,0.253984,0.492008,-1.223457,-0.551552,-1.223457,-0.125506,-0.055876


In [50]:
#Crossvalidation K-fold

kf = KFold(n_splits = 10, shuffle = False, random_state = None)
result = next(kf.split(fit_df), None)

train = fit_df.iloc[result[0]]
test =  fit_df.iloc[result[1]]

X_train = train.iloc[:, 1:3]
y_train = train[['pathBTS1',
                 'pathBTS2',
                 'pathBTS3',
                 'pathBTS4',
                 'pathBTS5',
                 'pathBTS6']]


X_test= test.iloc[:, 1:3]
y_test = test[['pathBTS1',
               'pathBTS2',
               'pathBTS3',
               'pathBTS4',
               'pathBTS5',
               'pathBTS6']]



In [52]:
from util.MultiRegressor import MultiRegressor 
from sklearn.metrics import mean_squared_error
from util.locdefs import get_distance_in_meters, geodesicDistance
from math import ceil

clf = neighbors.KNeighborsRegressor()
parameters = {'n_neighbors':list(range(1,30))}

multiregressor = MultiRegressor(clf, parameters)
multiregressor.fit(X_train, y_train)

print(multiregressor.best_parameters_)

y_pred = multiregressor.predict(X_test)

#display(y_pred)
ems = mean_squared_error(y_test, y_pred)
print("Error pathBTSs", ems)

#list_temp = []
#for i in range(y_test.shape[0]): 
 #   list_temp.append(get_distance_in_meters(y_pred[i][0], y_test["lat"][i], y_pred[i][1], y_test["lon"][i])/1000)
    #list_temp.append(geodesicDistance((y_pred[i][0], y_test["lat"][i]), (y_pred[i][1], y_test["lon"][i])))                

#ems_dist = mean_squared_error([0]*y_test.shape[0], list_temp)
#print("EMS with data in meters ", ems_dist)


[{'n_neighbors': 16}, {'n_neighbors': 3}, {'n_neighbors': 4}, {'n_neighbors': 28}, {'n_neighbors': 11}, {'n_neighbors': 4}]
Error pathBTSs 0.11752931887817379


# Generate Grid


In [63]:
max_lat = np.array(fit_df[["lat"]]).max()
min_lat = np.array(fit_df[["lat"]]).min()

max_lon = np.array(fit_df[["lon"]]).max()
min_lon = np.array(fit_df[["lon"]]).min()

left_up = (max_lat, min_lon)
right_up = (max_lat, max_lon)
left_down = (min_lat, min_lon)
right_down = (min_lat, max_lon)

#left_side = get_distance_in_meters(left_up[0], left_down[0], left_up[1], left_down[1])
#right_side = get_distance_in_meters(right_up[0], right_down[0], right_up[1], right_down[1])

left_side = geodesicDistance(left_up, left_down)
right_side = geodesicDistance(right_up, right_down)

#top_side = get_distance_in_meters(left_up[0], left_up[1], right_up[0], right_up[1])
#bottom_side = get_distance_in_meters(left_down[0], left_down[1], right_down[0], right_down[1])

top_side = geodesicDistance(left_up, right_up)
bottom_side = geodesicDistance(left_down, right_down)


side_a = max(left_side, right_side) #in km
side_b = max(top_side, bottom_side) #in km

print("Region dimensions in Km ", side_a, side_b)

cell_size = 0.020 #grid in kilometers
side_square = max(side_a, side_b)

grid_step = ceil(side_square / cell_size) + 1
grid_lats = np.linspace(min_lat, max_lat, grid_step)
grid_lons = np.linspace(min_lon, max_lon, grid_step)

lat_centers = [(l1 + l2)/2 for l1, l2 in zip(grid_lats, grid_lats[1:])]
lon_centers = [(l1 + l2)/2 for l1, l2 in zip(grid_lons, grid_lons[1:])]

samples = []
for lat in lat_centers:
    for lon in lon_centers:
        samples.append((lat, lon))

samples_df = pd.DataFrame({'lat' : [x[0] for x in samples], 'lon': [x[1] for x in samples]})
path_loss_predict = multiregressor.predict(samples_df)

Region dimensions in Km  2.0273225460434894 2.3950594892869215
